# Set up

In [1]:
import os
import json
import yaml
import shutil
import zipfile
import random

# Define labels

In [3]:
base_dir = os.getcwd()
print(base_dir)

/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code/annot


In [5]:
!ls ../../cvat_export

export_0122_0921  export_0122_0921.zip


In [ ]:
base_dir = "../.."

In [11]:
dir = base_dir + "/cvat_export/export_0122_0921"

filenames = [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames))
#print(filenames)

FileNotFoundError: [Errno 2] No such file or directory: '/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code/annot/cvat_export/export_0122_0921'

In [12]:
filenames = sorted(filenames)
filenames

['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499.jpg',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414.jpg',
 'signid_1000-mutcd_R7-20-location_40.777607153,-73.893456326-heading_253.524.jpg',
 'signid_1000-mutcd_R7-20-location_40.77764749,-73.893514257-heading_201.477.jpg',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59.jpg',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066.jpg',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412.jpg',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248.jpg',
 'signid_10002-mutcd_R7-51-location_40.712959179,-73.907034457-heading_218.969.jpg',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976.jpg',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528.jpg',
 'signid_10003-mutcd_R7-51-location_40.713395939,-73.913896026-heading_153.92

In [3]:
with open("asset/filenames.txt", "w") as f:
    for filename in filenames:
        f.write(f"{filename}\n")

In [4]:
# Define function to extract category from filename
def extract_category(filename):
    category = filename.split("-")[2]
    category = "R7-" + category
    return category

In [5]:
extract_category('signid_1473-mutcd_R7-20-location_40.790013029,-73.797457291-heading_109.833.jpg')

'R7-20'

In [6]:
# Generate JSON for CVAT auto-labeling
annotations = {
    "labels": [],
    "shapes": []
}

for filename in filenames:
    if filename.endswith((".jpg", ".png")):  # Adjust for your file type
        category = extract_category(filename)
        annotations["labels"].append({"name": category, "attributes": []})

# Save the pre-labeling JSON
with open("asset/pre_label_annotations.json", "w") as f:
    json.dump(annotations, f)

print("Pre-labeling JSON saved!")

Pre-labeling JSON saved!


In [7]:
import json

# Read the filenames from your provided file
filename_path = "asset/filenames.txt"
with open(filename_path, "r") as f:
    filenames = [line.strip() for line in f.readlines()]

# Function to extract category from filename
def extract_category(filename):
    category = filename.split("-")[2]  # Extract the category part (e.g., "R7-1")
    return "R7-" + category  # Add "R7-" prefix if not already present

# Initialize COCO JSON structure
coco_data = {
    "categories": [],
    "images": [],
    "annotations": []
}

# Map for unique category IDs
category_map = {}
category_id = 1  # Start category IDs at 1
annotation_id = 1  # Start annotation IDs at 1

# Process each filename to populate the COCO structure
for image_id, filename in enumerate(filenames):
    # Extract category
    category_name = extract_category(filename)

    # Add category to the categories list if not already added
    if category_name not in category_map:
        category_map[category_name] = category_id
        coco_data["categories"].append({
            "id": category_id,
            "name": category_name
        })
        category_id += 1

    # Add image metadata
    coco_data["images"].append({
        "id": image_id,
        "file_name": filename,
        "width": 640,  # Placeholder (set actual width if available)
        "height": 640  # Placeholder (set actual height if available)
    })

    # Add empty annotation (bounding box is blank)
    coco_data["annotations"].append({
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_map[category_name],
        "bbox": [100,100,300,300],  # Empty bounding box
        "area": 0,  # Placeholder (area calculation requires bbox)
        "iscrowd": 0,  # Default to 0 (not crowded)
        "segmentation": [] # empty segmentation
    })
    annotation_id += 1

# Save COCO JSON to file
output_path = "asset/coco_annotations.json"
with open(output_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print(f"COCO annotations saved to {output_path}!")

COCO annotations saved to asset/coco_annotations.json!


# Preload known annotations 

# Work within CVAT

# Import exported annotations

## set up

In [2]:
base_dir = os.getcwd()
print(base_dir)

/Users/jaeglee/ongoings/parking_sign/nyc/codes


In [11]:
base_dir = "./../cvat_exports/export_0115_2116_0"
print(base_dir)

./../cvat_exports/export_0115_2116_0


## use only the part that is finished

In [26]:
folder_path = os.path.join(base_dir, "images")

filenames_img = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames_img))

filenames_img = sorted(filenames_img)
filenames_img = filenames_img[0:1685]
print(len(filenames_img))
filenames_img

4974
1685


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499.jpg',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414.jpg',
 'signid_1000-mutcd_R7-20-location_40.777607153,-73.893456326-heading_253.524.jpg',
 'signid_1000-mutcd_R7-20-location_40.77764749,-73.893514257-heading_201.477.jpg',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59.jpg',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066.jpg',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412.jpg',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248.jpg',
 'signid_10002-mutcd_R7-51-location_40.712959179,-73.907034457-heading_218.969.jpg',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976.jpg',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528.jpg',
 'signid_10003-mutcd_R7-51-location_40.713395939,-73.913896026-heading_153.92

In [27]:
filenames_img = [f.split(".j")[0] for f in filenames_img]
filenames_img

['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414',
 'signid_1000-mutcd_R7-20-location_40.777607153,-73.893456326-heading_253.524',
 'signid_1000-mutcd_R7-20-location_40.77764749,-73.893514257-heading_201.477',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248',
 'signid_10002-mutcd_R7-51-location_40.712959179,-73.907034457-heading_218.969',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528',
 'signid_10003-mutcd_R7-51-location_40.713395939,-73.913896026-heading_153.923',
 'signid_10004-mutcd_R7-51-location_40.7

In [32]:
folder_path = os.path.join(base_dir, "labels")

filenames_lbl = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith(".txt")]
filenames_lbl = sorted(filenames_lbl)
print(len(filenames_lbl))
filenames_lbl

3858


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499.txt',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414.txt',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59.txt',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066.txt',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412.txt',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248.txt',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976.txt',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528.txt',
 'signid_10006-mutcd_R7-51-location_40.713302211,-73.912521961-heading_154.039.txt',
 'signid_10007-mutcd_R7-51-location_40.713280266,-73.912187534-heading_152.429.txt',
 'signid_10009-mutcd_R7-51-location_40.713226346,-73.911350267-heading_219.809.txt',
 'signid_10009-mutcd_R7-51-location_40.713231422,-73.911430166-heading_151

In [41]:
filenames_lbl = [f for f in filenames_lbl if f.split(".t")[0] in filenames_img]
filenames_lbl = [f.split(".t")[0] for f in filenames_lbl]
print(len(filenames_lbl))
filenames_lbl

569


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528',
 'signid_10006-mutcd_R7-51-location_40.713302211,-73.912521961-heading_154.039',
 'signid_10007-mutcd_R7-51-location_40.713280266,-73.912187534-heading_152.429',
 'signid_10009-mutcd_R7-51-location_40.713226346,-73.911350267-heading_219.809',
 'signid_10009-mutcd_R7-51-location_40.713231422,-73.911430166-heading_151.765',
 'signid_10011-mutcd_R7-51-location_4

In [43]:
filenames_img = [f for f in filenames_img if f in filenames_lbl]
print(len(filenames_img))
filenames_img

569


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528',
 'signid_10006-mutcd_R7-51-location_40.713302211,-73.912521961-heading_154.039',
 'signid_10007-mutcd_R7-51-location_40.713280266,-73.912187534-heading_152.429',
 'signid_10009-mutcd_R7-51-location_40.713226346,-73.911350267-heading_219.809',
 'signid_10009-mutcd_R7-51-location_40.713231422,-73.911430166-heading_151.765',
 'signid_10011-mutcd_R7-51-location_4

In [46]:
print(len(filenames_img))
filenames_img

569


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528',
 'signid_10006-mutcd_R7-51-location_40.713302211,-73.912521961-heading_154.039',
 'signid_10007-mutcd_R7-51-location_40.713280266,-73.912187534-heading_152.429',
 'signid_10009-mutcd_R7-51-location_40.713226346,-73.911350267-heading_219.809',
 'signid_10009-mutcd_R7-51-location_40.713231422,-73.911430166-heading_151.765',
 'signid_10011-mutcd_R7-51-location_4

In [47]:
print(len(filenames_lbl))
filenames_lbl

569


['signid_100-mutcd_R7-1-location_40.849854989,-73.89828847-heading_81.499',
 'signid_100-mutcd_R7-1-location_40.849914685,-73.898255013-heading_139.414',
 'signid_10000-mutcd_R7-51-location_40.712987445,-73.907559555-heading_222.59',
 'signid_10000-mutcd_R7-51-location_40.712991965,-73.907642327-heading_147.066',
 'signid_10001-mutcd_R7-51-location_40.712980208,-73.907427025-heading_221.412',
 'signid_10001-mutcd_R7-51-location_40.712984702,-73.907509318-heading_147.248',
 'signid_10002-mutcd_R7-51-location_40.712963352,-73.907114597-heading_149.976',
 'signid_10003-mutcd_R7-51-location_40.713391005,-73.913818281-heading_217.528',
 'signid_10006-mutcd_R7-51-location_40.713302211,-73.912521961-heading_154.039',
 'signid_10007-mutcd_R7-51-location_40.713280266,-73.912187534-heading_152.429',
 'signid_10009-mutcd_R7-51-location_40.713226346,-73.911350267-heading_219.809',
 'signid_10009-mutcd_R7-51-location_40.713231422,-73.911430166-heading_151.765',
 'signid_10011-mutcd_R7-51-location_4

In [49]:
folder_path = os.path.join(base_dir, "images")
files_to_keep = filenames_img

# Iterate through all files in the directory
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the current file is not in the keep list
    if file_name.split(".j")[0] not in files_to_keep:
        # Delete the file
        if os.path.isfile(file_path):  # Ensure it's a file, not a directory
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        else:
            print(f"Skipped: {file_path} (not a file)")

Deleted: ./../cvat_exports/export_0115_2116/images/signid_12119-mutcd_R7-61-location_40.676000654,-73.855420118-heading_235.679.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_783-mutcd_R7-20-location_40.559768773,-74.169626469-heading_277.725.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_9842-mutcd_R7-45-location_40.702087225,-73.882951022-heading_107.954.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_3711-mutcd_R7-40-location_40.627519763,-74.080929563-heading_41.191.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_10050-mutcd_R7-57-location_40.717301728,-73.784823289-heading_224.079.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_8177-mutcd_R7-40-location_40.681059854,-73.968113609-heading_231.932.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_820-mutcd_R7-20-location_40.569150954,-74.124602648-heading_285.031.jpg
Deleted: ./../cvat_exports/export_0115_2116/images/signid_13463-mutcd_R7-88-location

In [ ]:
folder_path = os.path.join(base_dir, "labels")
files_to_keep = filenames_lbl

# Iterate through all files in the directory
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the current file is not in the keep list
    if file_name.split(".t")[0] not in files_to_keep:
        # Delete the file
        if os.path.isfile(file_path):  # Ensure it's a file, not a directory
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        else:
            print(f"Skipped: {file_path} (not a file)")

## split

In [12]:
# Folder to clean up
folder_path = os.path.join(base_dir, "images")  
# Traverse the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.startswith('._'):
            file_path = os.path.join(root, file)
            try:
                os.remove(file_path)
                #print(f"Removed: {file_path}")
            except Exception as e:
                print(f"Error removing {file_path}: {e}")

# Folder to clean up
folder_path = os.path.join(base_dir, "labels")
# Traverse the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.startswith('._'):
            file_path = os.path.join(root, file)
            try:
                os.remove(file_path)
                #print(f"Removed: {file_path}")
            except Exception as e:
                print(f"Error removing {file_path}: {e}")

In [13]:
folder_path = os.path.join(base_dir, "images")  

filenames_img = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
filenames_img = sorted(filenames_img)
print(len(filenames_img))

filenames_img = [f for f in os.listdir(folder_path) if not f.startswith('._')]
filenames_img = [f.split(".j")[0] for f in filenames_img]

random.seed(0)
random.shuffle(filenames_img)

# Calculate the split index
split_between = int(len(filenames_img) * 0.8)

# Split into training and testing sets
train_ids = filenames_img[:split_between]
val_ids = filenames_img[split_between:]
print(train_ids)
print(val_ids)

569
['signid_1985-mutcd_R7-20-location_40.718153498,-73.823942853-heading_121.449', 'signid_11863-mutcd_R7-61-location_40.679056193,-73.791613127-heading_277.719', 'signid_2205-mutcd_R7-20-location_40.608655971,-74.133127152-heading_112.122', 'signid_2285-mutcd_R7-20-location_40.590165729,-74.098064222-heading_87.674', 'signid_11097-mutcd_R7-61-location_40.674852201,-73.806927466-heading_199.563', 'signid_1869-mutcd_R7-20-location_40.579467035,-73.838246863-heading_291.087', 'signid_2444-mutcd_R7-28-location_40.643884893,-73.951740479-heading_244.988', 'signid_2295-mutcd_R7-20-location_40.562852648,-74.114588742-heading_198.632', 'signid_13401-mutcd_R7-61-location_40.601667682,-74.08194088-heading_91.541', 'signid_10439-mutcd_R7-61-location_40.572137916,-73.857441478-heading_13.52', 'signid_2349-mutcd_R7-22-location_40.762228126,-73.902308989-heading_109.457', 'signid_12132-mutcd_R7-61-location_40.65560156,-73.828476086-heading_324.518', 'signid_13446-mutcd_R7-88-location_40.732039168,

In [14]:
base_dir

'./../cvat_exports/export_0115_2116_0'

In [15]:
# List of filenames to filter
# Define source and target folders
source_folder = os.path.join(base_dir, "images")  
target_folders = [os.path.join(source_folder, "train"), 
                  os.path.join(source_folder, "val")]
expander = ".jpg"
sets = [train_ids, val_ids]

for i in range(2):
    set_i = sets[i]
    target_folder_i = target_folders[i]
    
    set_i = [f + expander for f in set_i]

    # Create target folder if it does not exist
    os.makedirs(target_folder_i, exist_ok=True)

    # Loop through each filename in the list and move it if it exists
    for f in set_i:
        source_file = os.path.join(source_folder, f)
        target_file = os.path.join(target_folder_i, f)
    
        # Check if the file exists in the source folder
        if os.path.isfile(source_file):
            shutil.move(source_file, target_file)
            #print(f"Copied: {filename}")
        #else:
            #print(f"File not found: {filename}")

# Path to the directory
root_dir = source_folder

# Traverse the directory recursively
for root, dirs, files in os.walk(root_dir):
    num_files = len(files)  # Count files in the current folder
    folder_name = os.path.relpath(root, root_dir)  # Relative folder path
    print(f"Folder: {folder_name}, Number of files: {num_files}")

Folder: ., Number of files: 1
Folder: train, Number of files: 455
Folder: val, Number of files: 114


In [17]:
# List of filenames to filter
# Define source and target folders
source_folder = os.path.join(base_dir, "labels")  
target_folders = [os.path.join(source_folder, "train"), 
                  os.path.join(source_folder, "val")]
expander = ".txt"
sets = [train_ids, val_ids]

for i in range(2):
    set_i = sets[i]
    target_folder_i = target_folders[i]
    
    set_i = [f + expander for f in set_i]

    # Create target folder if it does not exist
    os.makedirs(target_folder_i, exist_ok=True)

    # Loop through each filename in the list and move it if it exists
    for f in set_i:
        source_file = os.path.join(source_folder, f)
        target_file = os.path.join(target_folder_i, f)
    
        # Check if the file exists in the source folder
        if os.path.isfile(source_file):
            shutil.move(source_file, target_file)
            #print(f"Copied: {filename}")
        #else:
            #print(f"File not found: {filename}")

# Path to the directory
root_dir = source_folder

# Traverse the directory recursively
for root, dirs, files in os.walk(root_dir):
    num_files = len(files)  # Count files in the current folder
    folder_name = os.path.relpath(root, root_dir)  # Relative folder path
    print(f"Folder: {folder_name}, Number of files: {num_files}")

Folder: ., Number of files: 1
Folder: train, Number of files: 455
Folder: val, Number of files: 114


# Check distribution of classes after annotation

In [3]:
mutcd_dict = {
    0: "empty-so-disregard",   # may have to remove this
    1: "R7-1",
    2: "R7-20",
    3: "R7-21",
    4: "R7-22",
    5: "R7-24",
    6: "R7-27",
    7: "R7-28",
    8: "R7-30",
    9: "R7-31",
    10: "R7-32",
    11: "R7-34",
    12: "R7-40",
    13: "R7-41",
    14: "R7-42",
    15: "R7-43",
    16: "R7-44",
    17: "R7-45",
    18: "R7-49",
    19: "R7-50",
    20: "R7-51",
    21: "R7-55",
    22: "R7-56",
    23: "R7-57",
    24: "R7-59",
    25: "R7-61",
    26: "R7-65",
    27: "R7-66",
    28: "R7-68",
    29: "R7-69",
    30: "R7-70",
    31: "R7-88",
    32: "R7-100",
    33: "R7-101",
    34: "R7-110",
    35: "R7-125",
    36: "R7-128",
    37: "R7-182",
    38: "truck-loading-only",
    39: "authorized-vehicle-only"
  }

In [5]:
current_folder = os.path.join(base_dir, "labels") 

label_folders = [os.path.join(current_folder, "train"), 
                 os.path.join(current_folder, "val"), 
                 os.path.join(current_folder, "test")]

distinct_label_list = list()

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            lines4outfile = []
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    #print(parts[0], mutcd_dict[int(parts[0])])
                    distinct_label_list.append(str(mutcd_dict[int(parts[0])]))
distinct_label_list

['R7-22',
 'R7-43',
 'R7-88',
 'R7-1',
 'R7-70',
 'R7-70',
 'R7-49',
 'R7-88',
 'R7-57',
 'R7-20',
 'R7-20',
 'R7-88',
 'R7-61',
 'R7-28',
 'R7-51',
 'R7-22',
 'R7-20',
 'R7-20',
 'R7-61',
 'R7-61',
 'R7-20',
 'R7-88',
 'R7-20',
 'R7-88',
 'R7-22',
 'R7-22',
 'R7-88',
 'R7-61',
 'R7-61',
 'R7-20',
 'R7-1',
 'R7-1',
 'R7-61',
 'R7-61',
 'R7-40',
 'R7-61',
 'R7-1',
 'truck-loading-only',
 'R7-20',
 'R7-20',
 'R7-70',
 'R7-20',
 'R7-49',
 'R7-20',
 'R7-61',
 'R7-61',
 'R7-30',
 'R7-61',
 'R7-61',
 'R7-22',
 'R7-61',
 'R7-51',
 'R7-61',
 'R7-50',
 'R7-51',
 'R7-128',
 'R7-128',
 'R7-20',
 'R7-20',
 'R7-1',
 'R7-61',
 'R7-22',
 'R7-20',
 'R7-1',
 'R7-61',
 'R7-61',
 'R7-57',
 'R7-20',
 'R7-61',
 'R7-24',
 'R7-40',
 'R7-20',
 'R7-61',
 'R7-57',
 'R7-20',
 'R7-45',
 'R7-20',
 'R7-61',
 'R7-40',
 'R7-61',
 'R7-61',
 'R7-20',
 'R7-61',
 'R7-61',
 'R7-20',
 'R7-61',
 'R7-20',
 'R7-50',
 'R7-20',
 'R7-20',
 'R7-61',
 'R7-22',
 'R7-40',
 'R7-61',
 'R7-61',
 'R7-20',
 'R7-20',
 'R7-20',
 'R7-20',
 

In [6]:
from collections import Counter

# List with repeated elements

# Count frequencies
frequency_count = Counter(distinct_label_list)

# Print the frequencies
for element, count in frequency_count.items():
    print(f"{element}: {count}")

R7-22: 15
R7-43: 6
R7-88: 42
R7-1: 35
R7-70: 5
R7-49: 12
R7-57: 24
R7-20: 182
R7-61: 187
R7-28: 16
R7-51: 16
R7-40: 21
truck-loading-only: 3
R7-30: 1
R7-50: 3
R7-128: 11
R7-24: 21
R7-45: 8
R7-27: 8
R7-125: 2
R7-42: 1
R7-44: 1
R7-21: 2
authorized-vehicle-only: 1


# Prep for NC-1 model

In [64]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_1"

shutil.copytree(current_folder, target_folder)
print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

FileExistsError: [Errno 17] File exists: './../cvat_exports/export_0115_2116/labels_nc_1'

In [ ]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_1"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            lines4outfile = []
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    parts[0] = "0"  # Change the first part (ID) to 0
                    lines4outfile.append(" ".join(parts))  # Rejoin the line
        
        # Save the modified content back to the same file
        with open(file_path, "w") as outfile:
            lines4outfile = "\n".join(lines4outfile)
            outfile.write(lines4outfile.rstrip("\n"))

In [66]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_1"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for i, name in enumerate(txt_filenames[:5]):
        # File path
        file_path = os.path.join(label_folder, name) 
        print(f"{i+1}th: {file_path}")
        # Read, modify, and overwrite the file
        with open(file_path, "r") as file:
            data = file.read()
            print(data)
    print("\nnext group following: ")

1th: ./../cvat_exports/export_0115_2116/labels_nc_1/train/signid_2350-mutcd_R7-22-location_40.723151738,-73.90875041-heading_42.95.txt
0 0.695992 0.494359 0.018859 0.016531
2th: ./../cvat_exports/export_0115_2116/labels_nc_1/train/signid_12123-mutcd_R7-61-location_40.771013669,-73.903444511-heading_9.374.txt
0 0.345133 0.385930 0.060984 0.045641
3th: ./../cvat_exports/export_0115_2116/labels_nc_1/train/signid_13470-mutcd_R7-88-location_40.733787748,-73.815204383-heading_210.536.txt
0 0.436703 0.504469 0.051594 0.023562
4th: ./../cvat_exports/export_0115_2116/labels_nc_1/train/signid_207-mutcd_R7-1-location_40.713557025,-73.782494654-heading_293.989.txt
0 0.021711 0.285742 0.036266 0.088828
5th: ./../cvat_exports/export_0115_2116/labels_nc_1/train/signid_13408-mutcd_R7-70-location_40.902537124,-73.854577911-heading_153.915.txt
0 0.380758 0.348156 0.099109 0.071188

next group following: 
1th: ./../cvat_exports/export_0115_2116/labels_nc_1/val/signid_10025-mutcd_R7-57-location_40.7137300

# Prep for NC-2 model

In [39]:
base_dir = "./../cvat_exports/export_0115_2116_0"
print(base_dir)

./../cvat_exports/export_0115_2116_0


In [70]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_2"

shutil.copytree(current_folder, target_folder)
print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

Copied folder from './../cvat_exports/export_0115_2116/labels' to './../cvat_exports/export_0115_2116/labels_nc_2'.


In [1]:
# List of all 40 classes
classes = [
    "empty-so-disregard",
    "R7-1",
    "R7-20",
    "R7-21",
    "R7-22",
    "R7-24",
    "R7-27",
    "R7-28",
    "R7-30",
    "R7-31",
    "R7-32",
    "R7-34",
    "R7-40",
    "R7-41",
    "R7-42",
    "R7-43",
    "R7-44",
    "R7-45",
    "R7-49",
    "R7-50",
    "R7-51",
    "R7-55",
    "R7-56",
    "R7-57",
    "R7-59",
    "R7-61",
    "R7-65",
    "R7-66",
    "R7-68",
    "R7-69",
    "R7-70",
    "R7-88",
    "R7-100",
    "R7-101",
    "R7-110",
    "R7-125",
    "R7-128",
    "R7-182",
    "truck-loading-only",
    "authorized-vehicle-only"
]

# Set of permissive classes
permissive_classes = {"R7-125", "R7-128"}

# Mapping function
def map_to_category(class_name):
    if class_name in permissive_classes:
        return "permissive"
    else:
        return "prohibitive"

# Test the mapping function
mapped_classes = {cls: map_to_category(cls) for cls in classes}

# Print the mapping
for cls, category in mapped_classes.items():
    print(f"{cls}: {category}")

empty-so-disregard: prohibitive
R7-1: prohibitive
R7-20: prohibitive
R7-21: prohibitive
R7-22: prohibitive
R7-24: prohibitive
R7-27: prohibitive
R7-28: prohibitive
R7-30: prohibitive
R7-31: prohibitive
R7-32: prohibitive
R7-34: prohibitive
R7-40: prohibitive
R7-41: prohibitive
R7-42: prohibitive
R7-43: prohibitive
R7-44: prohibitive
R7-45: prohibitive
R7-49: prohibitive
R7-50: prohibitive
R7-51: prohibitive
R7-55: prohibitive
R7-56: prohibitive
R7-57: prohibitive
R7-59: prohibitive
R7-61: prohibitive
R7-65: prohibitive
R7-66: prohibitive
R7-68: prohibitive
R7-69: prohibitive
R7-70: prohibitive
R7-88: prohibitive
R7-100: prohibitive
R7-101: prohibitive
R7-110: prohibitive
R7-125: permissive
R7-128: permissive
R7-182: prohibitive
truck-loading-only: prohibitive
authorized-vehicle-only: prohibitive


In [8]:
mapped_classes

{'empty-so-disregard': 'prohibitive',
 'R7-1': 'prohibitive',
 'R7-20': 'prohibitive',
 'R7-21': 'prohibitive',
 'R7-22': 'prohibitive',
 'R7-24': 'prohibitive',
 'R7-27': 'prohibitive',
 'R7-28': 'prohibitive',
 'R7-30': 'prohibitive',
 'R7-31': 'prohibitive',
 'R7-32': 'prohibitive',
 'R7-34': 'prohibitive',
 'R7-40': 'prohibitive',
 'R7-41': 'prohibitive',
 'R7-42': 'prohibitive',
 'R7-43': 'prohibitive',
 'R7-44': 'prohibitive',
 'R7-45': 'prohibitive',
 'R7-49': 'prohibitive',
 'R7-50': 'prohibitive',
 'R7-51': 'prohibitive',
 'R7-55': 'prohibitive',
 'R7-56': 'prohibitive',
 'R7-57': 'prohibitive',
 'R7-59': 'prohibitive',
 'R7-61': 'prohibitive',
 'R7-65': 'prohibitive',
 'R7-66': 'prohibitive',
 'R7-68': 'prohibitive',
 'R7-69': 'prohibitive',
 'R7-70': 'prohibitive',
 'R7-88': 'prohibitive',
 'R7-100': 'prohibitive',
 'R7-101': 'prohibitive',
 'R7-110': 'prohibitive',
 'R7-125': 'permissive',
 'R7-128': 'permissive',
 'R7-182': 'prohibitive',
 'truck-loading-only': 'prohibitiv

In [7]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_2"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

distinct_indx = []

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    #print(parts[0], end=",")
                    distinct_indx.append(int(parts[0]))
distinct_indx = set(distinct_indx)
print(distinct_indx)

{1, 2, 3, 4, 5, 6, 7, 8, 12, 14, 15, 16, 17, 18, 19, 20, 23, 25, 30, 31, 35, 36, 38, 39}


In [15]:
mapped_classes_keys = list(mapped_classes.keys())
mapped_classes_keys

['empty-so-disregard',
 'R7-1',
 'R7-20',
 'R7-21',
 'R7-22',
 'R7-24',
 'R7-27',
 'R7-28',
 'R7-30',
 'R7-31',
 'R7-32',
 'R7-34',
 'R7-40',
 'R7-41',
 'R7-42',
 'R7-43',
 'R7-44',
 'R7-45',
 'R7-49',
 'R7-50',
 'R7-51',
 'R7-55',
 'R7-56',
 'R7-57',
 'R7-59',
 'R7-61',
 'R7-65',
 'R7-66',
 'R7-68',
 'R7-69',
 'R7-70',
 'R7-88',
 'R7-100',
 'R7-101',
 'R7-110',
 'R7-125',
 'R7-128',
 'R7-182',
 'truck-loading-only',
 'authorized-vehicle-only']

In [16]:
mapped_classes[mapped_classes_keys[1]]

'prohibitive'

In [21]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_2"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

two_classes = {"permissive": 0,
               "prohibitive": 1}

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            lines4outfile = []
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    parts[0] = mapped_classes[mapped_classes_keys[int(parts[0])]]    # Change the first part (ID) to 0
                    parts[0] = str(parts[0])
                    parts[0] = str(two_classes[parts[0]])
                    lines4outfile.append(" ".join(parts))  # Rejoin the line

        #print(lines4outfile)

        # Save the modified content back to the same file
        with open(file_path, "w") as outfile:
            lines4outfile = "\n".join(lines4outfile)
            outfile.write(lines4outfile.rstrip("\n"))

['1 0.695992 0.494359 0.018859 0.016531']
['1 0.345133 0.385930 0.060984 0.045641']
['1 0.436703 0.504469 0.051594 0.023562']
['1 0.021711 0.285742 0.036266 0.088828']
['1 0.380758 0.348156 0.099109 0.071188']
['1 0.254477 0.358047 0.092922 0.076406']
['1 0.301914 0.352172 0.071484 0.053781']
['1 0.430547 0.460031 0.035156 0.046500']
['1 0.947016 0.475477 0.030906 0.025609']
['1 0.795055 0.489914 0.053016 0.036109']
['1 0.289172 0.446555 0.049281 0.032828']
['1 0.188211 0.403570 0.086703 0.061922']
['1 0.129406 0.476945 0.040875 0.135484']
['1 0.078242 0.469289 0.014141 0.047984']
['1 0.450703 0.411875 0.067031 0.045313']
['1 0.437836 0.427297 0.071297 0.057406']
['1 0.052156 0.446563 0.038187 0.025625']
['1 0.932555 0.452719 0.016359 0.033938']
['1 0.882383 0.458617 0.031328 0.068078']
['1 0.186711 0.425836 0.021578 0.040203']
['1 0.975625 0.200656 0.048750 0.124000']
['1 0.439289 0.488430 0.062984 0.045328']
['1 0.681414 0.422664 0.080641 0.054984']
['1 0.857992 0.127203 0.134422 0.1

In [22]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_2"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for i, name in enumerate(txt_filenames[:5]):
        # File path
        file_path = os.path.join(label_folder, name) 
        print(f"{i+1}th: {file_path}")
        # Read, modify, and overwrite the file
        with open(file_path, "r") as file:
            data = file.read()
            print(data)
    print("\nnext group following: ")

1th: ./../cvat_exports/export_0115_2116/labels_nc_2/train/signid_2350-mutcd_R7-22-location_40.723151738,-73.90875041-heading_42.95.txt
1 0.695992 0.494359 0.018859 0.016531
2th: ./../cvat_exports/export_0115_2116/labels_nc_2/train/signid_12123-mutcd_R7-61-location_40.771013669,-73.903444511-heading_9.374.txt
1 0.345133 0.385930 0.060984 0.045641
3th: ./../cvat_exports/export_0115_2116/labels_nc_2/train/signid_13470-mutcd_R7-88-location_40.733787748,-73.815204383-heading_210.536.txt
1 0.436703 0.504469 0.051594 0.023562
4th: ./../cvat_exports/export_0115_2116/labels_nc_2/train/signid_207-mutcd_R7-1-location_40.713557025,-73.782494654-heading_293.989.txt
1 0.021711 0.285742 0.036266 0.088828
5th: ./../cvat_exports/export_0115_2116/labels_nc_2/train/signid_13408-mutcd_R7-70-location_40.902537124,-73.854577911-heading_153.915.txt
1 0.380758 0.348156 0.099109 0.071188

next group following: 
1th: ./../cvat_exports/export_0115_2116/labels_nc_2/val/signid_10025-mutcd_R7-57-location_40.7137300

# Prep for NC-6 model

In [30]:
base_dir = "./../cvat_exports/export_0115_2116"
print(base_dir)

./../cvat_exports/export_0115_2116


In [18]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_6"

shutil.copytree(current_folder, target_folder)
print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

Copied folder from './../cvat_exports/export_0115_2116_0/labels' to './../cvat_exports/export_0115_2116_0/labels_nc_6'.


In [19]:
# List of all 40 classes
classes = [
    "empty-so-disregard",
    "R7-1",
    "R7-20",
    "R7-21",
    "R7-22",
    "R7-24",
    "R7-27",
    "R7-28",
    "R7-30",
    "R7-31",
    "R7-32",
    "R7-34",
    "R7-40",
    "R7-41",
    "R7-42",
    "R7-43",
    "R7-44",
    "R7-45",
    "R7-49",
    "R7-50",
    "R7-51",
    "R7-55",
    "R7-56",
    "R7-57",
    "R7-59",
    "R7-61",
    "R7-65",
    "R7-66",
    "R7-68",
    "R7-69",
    "R7-70",
    "R7-88",
    "R7-100",
    "R7-101",
    "R7-110",
    "R7-125",
    "R7-128",
    "R7-182",
    "truck-loading-only",
    "authorized-vehicle-only"
]

# Set of permissive classes
noStopping = {"R7-1"}
noStanding = {"R7-20",
                "R7-21",
                "R7-22",
                "R7-24",
                "R7-27",
                "R7-28",
                "R7-30",
                "R7-31",
                "R7-32"}
noParking = {    "R7-40",
                "R7-41",
                "R7-42",
                "R7-43",
                "R7-44",
                "R7-45",
                "R7-49",
                "R7-50",
                "R7-51",
                "R7-55",
                "R7-56",
                "R7-57",
                "R7-59",
                "R7-61",
                "R7-65",
                "R7-66",
                "R7-68",
                "R7-69",
                "R7-70",
                "R7-88",}
noStandingbusStop = {"R7-61"}
permissive = {"R7-125", "R7-128"}

In [20]:
# Mapping function with refactored logic
def map_to_category(class_name):
    # Define category mappings
    category_mapping = {
        "noStopping": noStopping,
        "noStanding": noStanding,
        "noParking": noParking,
        "noStandingbusStop": noStandingbusStop,
        "permissive": permissive
    }

    # Check the class_name against each category
    for category, class_set in category_mapping.items():
        if class_name in class_set:
            return category

    # Default return if no match is found
    return "unknown"

In [21]:
# Test the mapping function
mapped_classes = {cls: map_to_category(cls) for cls in classes}

# Print the mapping
mapped_classes

{'empty-so-disregard': 'unknown',
 'R7-1': 'noStopping',
 'R7-20': 'noStanding',
 'R7-21': 'noStanding',
 'R7-22': 'noStanding',
 'R7-24': 'noStanding',
 'R7-27': 'noStanding',
 'R7-28': 'noStanding',
 'R7-30': 'noStanding',
 'R7-31': 'noStanding',
 'R7-32': 'noStanding',
 'R7-34': 'unknown',
 'R7-40': 'noParking',
 'R7-41': 'noParking',
 'R7-42': 'noParking',
 'R7-43': 'noParking',
 'R7-44': 'noParking',
 'R7-45': 'noParking',
 'R7-49': 'noParking',
 'R7-50': 'noParking',
 'R7-51': 'noParking',
 'R7-55': 'noParking',
 'R7-56': 'noParking',
 'R7-57': 'noParking',
 'R7-59': 'noParking',
 'R7-61': 'noParking',
 'R7-65': 'noParking',
 'R7-66': 'noParking',
 'R7-68': 'noParking',
 'R7-69': 'noParking',
 'R7-70': 'noParking',
 'R7-88': 'noParking',
 'R7-100': 'unknown',
 'R7-101': 'unknown',
 'R7-110': 'unknown',
 'R7-125': 'permissive',
 'R7-128': 'permissive',
 'R7-182': 'unknown',
 'truck-loading-only': 'unknown',
 'authorized-vehicle-only': 'unknown'}

In [22]:
mapped_classes

{'empty-so-disregard': 'unknown',
 'R7-1': 'noStopping',
 'R7-20': 'noStanding',
 'R7-21': 'noStanding',
 'R7-22': 'noStanding',
 'R7-24': 'noStanding',
 'R7-27': 'noStanding',
 'R7-28': 'noStanding',
 'R7-30': 'noStanding',
 'R7-31': 'noStanding',
 'R7-32': 'noStanding',
 'R7-34': 'unknown',
 'R7-40': 'noParking',
 'R7-41': 'noParking',
 'R7-42': 'noParking',
 'R7-43': 'noParking',
 'R7-44': 'noParking',
 'R7-45': 'noParking',
 'R7-49': 'noParking',
 'R7-50': 'noParking',
 'R7-51': 'noParking',
 'R7-55': 'noParking',
 'R7-56': 'noParking',
 'R7-57': 'noParking',
 'R7-59': 'noParking',
 'R7-61': 'noParking',
 'R7-65': 'noParking',
 'R7-66': 'noParking',
 'R7-68': 'noParking',
 'R7-69': 'noParking',
 'R7-70': 'noParking',
 'R7-88': 'noParking',
 'R7-100': 'unknown',
 'R7-101': 'unknown',
 'R7-110': 'unknown',
 'R7-125': 'permissive',
 'R7-128': 'permissive',
 'R7-182': 'unknown',
 'truck-loading-only': 'unknown',
 'authorized-vehicle-only': 'unknown'}

In [ ]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_6"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val")]

distinct_indx = []

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    #print(parts[0], end=",")
                    distinct_indx.append(int(parts[0]))
distinct_indx = set(distinct_indx)
print(distinct_indx)

{1, 2, 3, 4, 5, 6, 7, 8, 12, 14, 15, 16, 17, 18, 19, 20, 23, 25, 30, 31, 35, 36, 38, 39}


In [25]:
mapped_classes_keys = list(mapped_classes.keys())
mapped_classes_keys

['empty-so-disregard',
 'R7-1',
 'R7-20',
 'R7-21',
 'R7-22',
 'R7-24',
 'R7-27',
 'R7-28',
 'R7-30',
 'R7-31',
 'R7-32',
 'R7-34',
 'R7-40',
 'R7-41',
 'R7-42',
 'R7-43',
 'R7-44',
 'R7-45',
 'R7-49',
 'R7-50',
 'R7-51',
 'R7-55',
 'R7-56',
 'R7-57',
 'R7-59',
 'R7-61',
 'R7-65',
 'R7-66',
 'R7-68',
 'R7-69',
 'R7-70',
 'R7-88',
 'R7-100',
 'R7-101',
 'R7-110',
 'R7-125',
 'R7-128',
 'R7-182',
 'truck-loading-only',
 'authorized-vehicle-only']

In [26]:
mapped_classes[mapped_classes_keys[1]]

'noStopping'

In [27]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_6"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val")]

five_classes = {"noStopping": 0,
                "noStanding": 1,
                "noParking": 2,
                "noStandingbusStop": 3,
                "permissive": 4,
                "unknown": 5}

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for name in txt_filenames:
        # File path
        file_path = os.path.join(label_folder, name) 
        
        # Read, modify, and overwrite the file
        with open(file_path, "r") as infile:
            lines4outfile = []
            for line in infile:
                parts = line.strip().split()  # Split the line into parts
                if parts:  # Ensure the line is not empty
                    parts[0] = mapped_classes[mapped_classes_keys[int(parts[0])]]   
                    print(parts[0])
                    parts[0] = str(parts[0])
                    print(parts[0])
                    parts[0] = str(five_classes[parts[0]])
                    lines4outfile.append(" ".join(parts))  # Rejoin the line

        #print(lines4outfile)

        # Save the modified content back to the same file
        with open(file_path, "w") as outfile:
            lines4outfile = "\n".join(lines4outfile)
            outfile.write(lines4outfile.rstrip("\n"))

noStanding
noStanding
noParking
noParking
noParking
noParking
noStopping
noStopping
noParking
noParking
noParking
noParking
noParking
noParking
noParking
noParking
noParking
noParking
noStanding
noStanding
noStanding
noStanding
noParking
noParking
noParking
noParking
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noStanding
noStanding
noParking
noParking
noParking
noParking
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noStanding
noStanding
noParking
noParking
noParking
noParking
noParking
noParking
noStanding
noStanding
noStopping
noStopping
noStopping
noStopping
noParking
noParking
noParking
noParking
noStanding
noStanding
noParking
noParking
noParking
noParking
noStopping
noStopping
unknown
unknown
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noParking
noParking
noStanding
noStanding
noParking
noParking
noParking
noParking
noStanding
noStanding
no

In [29]:
current_folder = os.path.join(base_dir, "labels") 
target_folder = current_folder + "_nc_6"

label_folders = [os.path.join(target_folder, "train"), 
                 os.path.join(target_folder, "val"), 
                 os.path.join(target_folder, "test")]

for label_folder in label_folders:
    txt_filenames = [f for f in os.listdir(label_folder) if (f.endswith('.txt') and not f.startswith('._'))]

    for i, name in enumerate(txt_filenames[:5]):
        # File path
        file_path = os.path.join(label_folder, name) 
        print(f"{i+1}th: {file_path}")
        # Read, modify, and overwrite the file
        with open(file_path, "r") as file:
            data = file.read()
            print(data)
    print("\nnext group following: ")

1th: ./../cvat_exports/export_0115_2116/labels_nc_5/train/signid_2350-mutcd_R7-22-location_40.723151738,-73.90875041-heading_42.95.txt
5 0.695992 0.494359 0.018859 0.016531
2th: ./../cvat_exports/export_0115_2116/labels_nc_5/train/signid_12123-mutcd_R7-61-location_40.771013669,-73.903444511-heading_9.374.txt
0 0.345133 0.385930 0.060984 0.045641
3th: ./../cvat_exports/export_0115_2116/labels_nc_5/train/signid_13470-mutcd_R7-88-location_40.733787748,-73.815204383-heading_210.536.txt
0 0.436703 0.504469 0.051594 0.023562
4th: ./../cvat_exports/export_0115_2116/labels_nc_5/train/signid_207-mutcd_R7-1-location_40.713557025,-73.782494654-heading_293.989.txt
5 0.021711 0.285742 0.036266 0.088828
5th: ./../cvat_exports/export_0115_2116/labels_nc_5/train/signid_13408-mutcd_R7-70-location_40.902537124,-73.854577911-heading_153.915.txt
1 0.380758 0.348156 0.099109 0.071188

next group following: 
1th: ./../cvat_exports/export_0115_2116/labels_nc_5/val/signid_10025-mutcd_R7-57-location_40.7137300